In [49]:
import tensorflow as tf
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input,Dense, Subtract, Add, Concatenate
from tensorflow.keras.optimizers import Adam

real_dem = pd.read_excel('C:/Users/ERIC/Desktop/Forecast Real Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',0)
forec_dem = pd.read_excel('C:/Users/ERIC/Desktop/Forecast Real Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',2)
'''
#Data from the forecast
cen_x_forec = forec_dem.iloc[:288,3:4]
sc = StandardScaler()
cen_x_train_forec = sc.fit_transform(cen_x_forec)
'''
# Training dataset
cen_x = real_dem.iloc[:, 3:7].values
cen_y = real_dem.iloc[:, 7:8].values

cen_x_train, cen_x_test, cen_y_train, cen_y_test = train_test_split(cen_x, cen_y, test_size=0.2, random_state=0)


sc = StandardScaler()
cen_x_train = sc.fit_transform(cen_x_train)
cen_x_test = sc.fit_transform(cen_x_test)

# Input from the current day
input_tensor = Input(shape=(cen_x.shape[1],))

hidden_layer = Dense(100, activation = 'relu')(input_tensor)
hidden_layer1 = Dense(50, activation = 'relu')(hidden_layer)
hidden_layer2 = Dense(25, activation = 'relu')(hidden_layer1)

output_tensor = Dense(1)(hidden_layer2)

model = Model(input_tensor, output_tensor)

model.compile(optimizer= Adam(0.01),  loss='mean_absolute_error', metrics = ['mean_absolute_error'])

model.summary()

model.fit(cen_x_train,
          cen_y_train,
          epochs=100,
          batch_size=2,
          validation_split=0.1,
          verbose=True)

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_110 (InputLayer)       [(None, 4)]               0         
_________________________________________________________________
dense_87 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_88 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_89 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_90 (Dense)             (None, 1)                 26        
Total params: 6,851
Trainable params: 6,851
Non-trainable params: 0
_________________________________________________________________
Train on 259 samples, validate on 29 samples
Epoch 1/100
259/259 [==============================] - 1s 4ms/sample - loss: 

Epoch 42/100
259/259 [==============================] - 0s 1ms/sample - loss: 107.9286 - mean_absolute_error: 107.9286 - val_loss: 71.4254 - val_mean_absolute_error: 71.4254
Epoch 43/100
259/259 [==============================] - 0s 1ms/sample - loss: 144.3058 - mean_absolute_error: 144.3058 - val_loss: 134.0013 - val_mean_absolute_error: 134.0013
Epoch 44/100
259/259 [==============================] - 0s 1ms/sample - loss: 115.7343 - mean_absolute_error: 115.7343 - val_loss: 49.3052 - val_mean_absolute_error: 49.3052
Epoch 45/100
259/259 [==============================] - 0s 1ms/sample - loss: 132.2774 - mean_absolute_error: 132.2774 - val_loss: 104.3699 - val_mean_absolute_error: 104.3699
Epoch 46/100
259/259 [==============================] - 0s 1ms/sample - loss: 80.5813 - mean_absolute_error: 80.5813 - val_loss: 81.4301 - val_mean_absolute_error: 81.4301
Epoch 47/100
259/259 [==============================] - 0s 1ms/sample - loss: 94.9827 - mean_absolute_error: 94.9827 - val_loss:

Epoch 89/100
259/259 [==============================] - 0s 1ms/sample - loss: 99.0068 - mean_absolute_error: 99.0068 - val_loss: 80.3750 - val_mean_absolute_error: 80.3750
Epoch 90/100
259/259 [==============================] - 0s 1ms/sample - loss: 105.5655 - mean_absolute_error: 105.5655 - val_loss: 107.4544 - val_mean_absolute_error: 107.4544
Epoch 91/100
259/259 [==============================] - 0s 2ms/sample - loss: 96.0333 - mean_absolute_error: 96.0333 - val_loss: 21.3499 - val_mean_absolute_error: 21.3499
Epoch 92/100
259/259 [==============================] - 0s 1ms/sample - loss: 101.6441 - mean_absolute_error: 101.6442 - val_loss: 121.0524 - val_mean_absolute_error: 121.0524
Epoch 93/100
259/259 [==============================] - 0s 2ms/sample - loss: 116.6535 - mean_absolute_error: 116.6534 - val_loss: 71.7990 - val_mean_absolute_error: 71.7990
Epoch 94/100
259/259 [==============================] - 0s 1ms/sample - loss: 93.6710 - mean_absolute_error: 93.6710 - val_loss: 1

In [47]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model.predict(cen_x_train)

print(np.sqrt(mean_squared_error(cen_y_train,pred_train)))
pres_train = pd.DataFrame(pred_train)
pres_train.columns = ['Prediction']
cen_y_train = pd.DataFrame(cen_y_train)
cen_y_train.columns = ['Real Value']
results = pd.concat([pres_train, cen_y_train], axis = 1)



#pred = model.predict(cen_x_test)
#print(np.sqrt(mean_squared_error(cen_y_test,pred)))
results                     

9.81698346099827


,Prediction,Real Value
0,7479.842285,7474.33949
1,5901.882812,5895.00295
2,5334.544434,5324.94201
3,7748.524902,7739.26681
4,7764.926758,7747.53124
...,...,...
283,6111.709473,6105.05688
284,6088.042969,6075.02433
285,7429.547363,7421.35666
286,6675.927246,6662.96869
